In [3]:
import  tensorflow as tf
from tensorflow.keras import datasets, layers, optimizers, Sequential, metrics

(xs, ys),_ = datasets.mnist.load_data()
print('datasets:', xs.shape, ys.shape, xs.min(), xs.max())

print(type(xs))
xs = tf.convert_to_tensor(xs, dtype=tf.float32) / 255.
db = tf.data.Dataset.from_tensor_slices((xs,ys))
# repeat repeats the whole dataset such that each value will appear count times
db = db.batch(32).repeat(10)

datasets: (60000, 28, 28) (60000,) 0 255
<class 'numpy.ndarray'>


In [20]:
network = Sequential([layers.Dense(256, activation='relu'),
                     layers.Dense(256, activation='relu'),
                     layers.Dense(256, activation='relu'),
                     layers.Dense(10)])

network.build(input_shape=(None, 28*28))
network.summary()

optimizer = optimizers.SGD(lr=0.01)
acc_meter = metrics.Accuracy()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               200960    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2570      
Total params: 335,114
Trainable params: 335,114
Non-trainable params: 0
_________________________________________________________________


In [21]:
for step, (x,y) in enumerate(db):

    with tf.GradientTape() as tape:
        # [b, 28, 28] => [b, 784]
        x = tf.reshape(x, (-1, 28*28))
        # [b, 784] => [b, 10]
        out = network(x)
        # [b] => [b, 10]
        y_onehot = tf.one_hot(y, depth=10)
        # [b, 10]
        loss = tf.square(out-y_onehot)
        # [b]
        loss = tf.reduce_sum(loss) / 32


    acc_meter.update_state(tf.argmax(out, axis=1), y)

    grads = tape.gradient(loss, network.trainable_variables)
    optimizer.apply_gradients(zip(grads, network.trainable_variables))


    if step % 200==0:

        print(step, 'loss:', float(loss), 'acc:', acc_meter.result().numpy())
        acc_meter.reset_states()

0 loss: 1.5306174755096436 acc: 0.0625
200 loss: 0.40297916531562805 acc: 0.67546874
400 loss: 0.4081514775753021 acc: 0.8453125
600 loss: 0.3423200845718384 acc: 0.8646875
800 loss: 0.24360646307468414 acc: 0.89390624
1000 loss: 0.30946558713912964 acc: 0.896875
1200 loss: 0.2788773775100708 acc: 0.9078125
1400 loss: 0.21690113842487335 acc: 0.91765624
1600 loss: 0.1752510368824005 acc: 0.9117187
1800 loss: 0.21858201920986176 acc: 0.92828125
2000 loss: 0.1999574601650238 acc: 0.9428125
2200 loss: 0.13363730907440186 acc: 0.93015623
2400 loss: 0.2297099530696869 acc: 0.9284375
2600 loss: 0.18208461999893188 acc: 0.9373438
2800 loss: 0.12813743948936462 acc: 0.9373438
3000 loss: 0.22194629907608032 acc: 0.93171877
3200 loss: 0.18575981259346008 acc: 0.94109374
3400 loss: 0.15277454257011414 acc: 0.9371875
3600 loss: 0.10971058905124664 acc: 0.936875
3800 loss: 0.16824674606323242 acc: 0.9560937
4000 loss: 0.18872258067131042 acc: 0.953125
4200 loss: 0.12394105643033981 acc: 0.9409375
4

In [32]:
t1 = tf.constant([0.1,0.1,0.2,0.6])
t2 = tf.constant([0,0,0,1.0])

In [36]:
l = tf.square(t1-t2)
print(l)

tf.Tensor([0.01       0.01       0.04       0.15999998], shape=(4,), dtype=float32)


In [38]:
tf.reduce_sum(l) / 1

<tf.Tensor: shape=(), dtype=float32, numpy=0.21999998>